In [2]:
import findspark
findspark.init()
import pyspark
import random
sc = pyspark.SparkContext(appName="Pi")

### Try using pyspark

In [3]:
file = open('amazon-meta.txt', 'r')
amazon = sc.textFile('amazon-meta.txt')

In [4]:
amazon.take(50)

['# Full information about Amazon Share the Love products',
 'Total items: 548552',
 '',
 'Id:   0',
 'ASIN: 0771044445',
 '  discontinued product',
 '',
 'Id:   1',
 'ASIN: 0827229534',
 '  title: Patterns of Preaching: A Sermon Sampler',
 '  group: Book',
 '  salesrank: 396585',
 '  similar: 5  0804215715  156101074X  0687023955  0687074231  082721619X',
 '  categories: 2',
 '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Preaching[12368]',
 '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Sermons[12370]',
 '  reviews: total: 2  downloaded: 2  avg rating: 5',
 '    2000-7-28  cutomer: A2JW67OY8U6HHK  rating: 5  votes:  10  helpful:   9',
 '    2003-12-14  cutomer: A2VE83MZF98ITY  rating: 5  votes:   6  helpful:   5',
 '',
 'Id:   2',
 'ASIN: 0738700797',
 '  title: Candlemas: Feast of Flames',
 '  group: Book',
 '  salesrank: 168596',
 '  similar: 5  0738700827  1567184960  1567182836  0738

### How to convert above data to df?

### Let's try using gzip: Works but takes long

In [5]:
import gzip
g = gzip.open("amazon-meta.txt.gz", 'rb')

In [6]:
#turns whole data into one list
data = []
for i in g:
    data.append(i)

In [7]:
#Create list of lists where every product is an inner list:
data_lists =[]
temp =[]
for i in data:
    if i[0:4]==b'\r\n':
        data_lists.append(temp)
        del temp
        temp =[]
    else:
        temp.append(i)

Let's extract each figure into its own list:

In [8]:
Id =[]
for i in data_lists[1:]:
    Id.append(i[0][6:])

In [9]:
ASIN =[]
for i in data_lists[1:]:
    ASIN.append(i[1][6:])

In [10]:
Title =[]
for i in data_lists[1:]:
    Title.append(i[2][9:])

In [11]:
Group =[]
for i in data_lists[1:]:
    try:
        Group.append(i[3][9:])
    except:
        Group.append(None)

In [12]:
Salesrank =[]
for i in data_lists[1:]:
    try:
        Salesrank.append(i[4][13:])
    except:
        Salesrank.append(None)

In [13]:
Similar = []
for i in data_lists[1:]:
    try:
        Similar.append(i[5][11:])
    except:
        Similar.append(None)

In [14]:
Categories =[]
for i in data_lists[1:]:
    try:
        Categories.append(i[6][14:])
    except:
        Categories.append(None)

In [15]:
temp =[]
Reviews =[]
for i in data_lists[1:]:
    try:
        temp.append(i[7])
        for x in i[7:]:
            if x[2:9]==b'reviews':
                Reviews.append(x)
    except:
        Reviews.append(None)

Now, let's join all the lists into a dataframe, and make sure NaN values are reflected into new df:

In [16]:
import pandas as pd
columns = ['Id']
df = pd.DataFrame(columns=columns)

In [17]:
df['Id'] = Id
df['Id'] = df['Id'].map(lambda x: str(x)[:-5])
df['Id'] = df['Id'].map(lambda x: str(x)[2:])
df['Id'] = df['Id'].map(lambda x: int(x))

In [18]:
df['ASIN'] = ASIN
df['ASIN'] = df['ASIN'].map(lambda x: str(x)[:-5])
df['ASIN'] = df['ASIN'].map(lambda x: str(x)[2:])

In [29]:
df['Title'] = Title
df['Title'] = df['Title'].map(lambda x: str(x)[:-5])
df['Title'] = df['Title'].map(lambda x: str(x)[2:])
Titles = []
for i in df['Title']:
    if str(i)=='inued product':
        Titles.append('discontinued product')
    else:
        Titles.append(str(i))
df['Title'] = Titles

In [61]:
df['Group'] = Group
df['Group'] = df['Group'].map(lambda x: str(x)[:-5])
df['Group'] = df['Group'].map(lambda x: str(x)[2:])
group =[]
for i in df['Group']:
    if i!= '':
        group.append(str(i))
    else:
        group.append(None)
df['Group'] = group

In [67]:
df['Salesrank'] = Salesrank
df['Salesrank'] = df['Salesrank'].map(lambda x: str(x)[:-5])
df['Salesrank'] = df['Salesrank'].map(lambda x: str(x)[2:])
salesrank = []
for i in df['Salesrank']:
    try:
        salesrank.append(int(i))
    except:
        salesrank.append(None)
df['Salesrank'] = salesrank

In [72]:
df['Similar'] = Similar
df['Similar'] = df['Similar'].map(lambda x: str(x)[:-5])
df['Similar'] = df['Similar'].map(lambda x: str(x)[2:])
similar = []
for i in df['Similar']:
    if i!='':
        similar.append(str(i))
    else:
        similar.append(None)
df['Similar'] = similar        

In [48]:
df['Categories'] = Categories
df['Categories'] = df['Categories'].map(lambda x: str(x)[:-5])
df['Categories'] = df['Categories'].map(lambda x: str(x)[2:])
categories = []
for i in df['Categories']:
    try:
        categories.append(int(i))
    except:
        categories.append(None)
df['Categories'] = categories

In [76]:
df['Reviews'] = Reviews
df['Reviews'] = df['Reviews'].map(lambda x: str(x)[:-5])
df['Reviews'] = df['Reviews'].map(lambda x: str(x)[2:])
reviews = []
for i in df['Reviews']:
    if i!='':
        reviews.append(str(i))
    else:
        reviews.append(None)
df['Reviews'] = reviews

In [77]:
df.head()

,Id,ASIN,Title,Group,Salesrank,Similar,Categories,Reviews
0,0,0771044445,discontinued product,None,NaN,None,NaN,None
1,1,0827229534,Patterns of Preaching: A Sermon Sampler,Book,396585.0,5 0804215715 156101074X 0687023955 0687074...,2.0,reviews: total: 2 downloaded: 2 avg rating: 5
2,2,0738700797,Candlemas: Feast of Flames,Book,168596.0,5 0738700827 1567184960 1567182836 0738700...,2.0,reviews: total: 12 downloaded: 12 avg rati...
3,3,0486287785,World War II Allied Fighter Planes Trading Cards,Book,1270652.0,0,1.0,reviews: total: 1 downloaded: 1 avg rating: 5
4,4,0842328327,Life Application Bible Commentary: 1 and 2 Tim...,Book,631289.0,5 0842328130 0830818138 0842330313 0842328...,5.0,reviews: total: 1 downloaded: 1 avg rating: 4


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548552 entries, 0 to 548551
Data columns (total 8 columns):
Id            548552 non-null int64
ASIN          548552 non-null object
Title         548552 non-null object
Group         542684 non-null object
Salesrank     542684 non-null float64
Similar       542684 non-null object
Categories    542684 non-null float64
Reviews       542684 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 33.5+ MB
